# Beijing Air-Quality Time Series Project
### Data Cleaning Notebook

#### by Dolci Sanders and Paul Torres


### Import Libraries

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import glob
from sklearn.model_selection import train_test_split
import pickle

### Read in Data

Our Data is provided by the UCI Machine Learning Repository
Beijing Multi-Site Air-Quality Data

Once Read in, we will look at the head and convert our time into data time. 

We have 12 data sets, one from each reporting site, to concatenate together to get the whole picture of Beijing's Air Quality. 


In [2]:
time = pd.read_csv('DATA/PRSA_Data_Tiantan_20130301-20170228.csv', index_col = None)

In [3]:
time.year.unique()

array([2013, 2014, 2015, 2016, 2017])

### Date Time Formatting
Crucial to time series, this must be set as the index. 

In [4]:
time['Date'] = pd.to_datetime(time[['year','month','day','hour']])
time = time.set_index('Date')

In [5]:
time.shape

(35064, 18)

### Dealing with NaN Values
After looking at the data, we wanted to look at null values and figure out what to do with these. 

We first calculated the mean and median for all of the values. These values varied wildly, and we were concerned they would have a negative effect on the predictions. 

However, upon further research, other methods have proven more effective in inputing the data for time series such as interpolating time. We elected to try imputing using the interpolate time method. 

Because we are doing univariate time series, we will not need to worry about the nan values of the other features.
--if we did want multivariate, this is still a great imputation option. 

In [6]:
time.isna().sum()

No            0
year          0
month         0
day           0
hour          0
PM2.5       677
PM10        597
SO2        1118
NO2         744
CO         1126
O3          843
TEMP         20
PRES         20
DEWP         20
RAIN         20
wd           78
WSPM         14
station       0
dtype: int64

In [7]:
time = time.interpolate(method='time')

### PM2.5 (Target Variable)
After interpolating using the time method, you can see this value now has no missing values. 

In [8]:
print('total PM2.5 Missing Values: ', time['PM2.5'].isna().sum())
print('mean: ', time['PM2.5'].mean())
print('median: ', time['PM2.5'].median())

total PM2.5 Missing Values:  0
mean:  82.03309662331738
median:  58.0


## Combine 12 Testing Site Tables
We have 12 testing sites with a table for each. 
So we built a function to automate our concatenation of all of these. 
For each test site, we loop through these, creating a whole dataframe for visualizations and also a separate loop of this to train test split, interpolate the test then the train using the time method.

Each of these will interpolate each table separately as shown with one table above, and then concatenate them for the most accurate results. 

The first will keep the whole data set, in case we want to do some additional visualizations. 
The second will be our test train split, which will be Date and PM2.5 values only. 
Thirdly we will leave out a set with stations still attached for further use at a later date. 


### Time DataFrame, import all as a whole, interpolate by table
Here we have the full data frame, not split with all the columns. 

In [15]:
# From the DATA folder, we wil run through all of the individual csv files.
path = r'DATA/'
allFiles = glob.glob(path + '/*.csv')

# Prep the test and train data frames.
time = pd.DataFrame()
times = []


for file_ in allFiles:
    
    # Read in and set data index.
    df = pd.read_csv(file_,index_col = None,header = 0)
    df['Date'] = pd.to_datetime(df[['year','month','day','hour']])
    df = df.set_index('Date')
    df = df.loc[:'2016-12-31 23:00:00']
    
    
    # Univariate time series, drop the other features
    df = df.drop(columns =['No'])
    

    # Interpolate the train data and add to the rest of the train list
    df = df.interpolate(method = 'time')
    times.append(df)
    
    
    
# Take the data frame we made at the top and combine the new data frame we processed here
time = pd.concat(times)
time = time.sort_values(['station','Date'])
time = time.resample('D').mean()
time.head()

,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,WSPM
Date,,,,,,,,,,,,,,,
2013-03-01,2013.0,3.0,1.0,11.5,7.326389,12.203125,9.182292,19.990602,404.861111,68.244792,1.173958,1026.569792,-20.148611,0.0,2.917708
2013-03-02,2013.0,3.0,2.0,11.5,31.475694,40.506944,31.757878,56.303763,939.715278,32.601112,0.260417,1026.552778,-16.271875,0.0,1.145139
2013-03-03,2013.0,3.0,3.0,11.5,79.291667,111.104167,49.286458,75.661458,1771.142361,22.788194,5.257292,1014.215278,-12.325347,0.0,1.463194
2013-03-04,2013.0,3.0,4.0,11.5,21.768569,40.359375,18.818692,40.877151,726.340278,56.833333,9.610764,1017.263194,-12.773264,0.0,2.227083
2013-03-05,2013.0,3.0,5.0,11.5,125.352959,159.236111,70.756944,103.337674,2021.434028,79.527778,6.630556,1010.551389,-7.916319,0.0,0.992014


### Univariate Train Test Split Interpolation with 80-20 Split

Because time series needs specific splits of the train and test, we have done this in the function. 
We have elected to use an 80-20 split with the last 20 being the test set for comparing the predictions. 

In [32]:
# From the DATA folder, we wil run through all of the individual csv files. 

path = r'DATA/'
allFiles = glob.glob(path + '/*.csv')

# Prep the test and train data frames.

test = pd.DataFrame()
train = pd.DataFrame()

trains = []
tests = []


for file_ in allFiles:
    
    # Read in and set data index. 
    df = pd.read_csv(file_,index_col = None,header = 0)
    df['Date'] = pd.to_datetime(df[['year','month','day','hour']])
    df = df.set_index('Date')
    #Limits to end 2016 
    df = df.loc[:'2016-12-31 23:00:00']
    df = df.drop(columns =['PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 
                           'DEWP', 'RAIN','wd', 'WSPM','No','year','month','day','hour'])

    df = df.resample('D').mean()
    
    # Drop unwanted columns, in univariate we should only have the date and the target. 
    
    # Train test split to prevent data leakage
    
    train = df[:int(df.shape[0]*0.8)]
    test = df[int(df.shape[0]*0.8):]
    
    
    # Interpolate the train data and add to the rest of the train list 
    
    train = train.interpolate(method = 'time')
    train['PM2.5'] = train['PM2.5'].fillna(method = 'bfill')
    trains.append(train)
    
    # Interpolate the test data and add to the rest of the test list
    
    test = test.interpolate(method = 'time')
    test['PM2.5'] = test['PM2.5'].fillna(method = 'bfill')
    tests.append(test)
    
    
# Take the data fame we made at the top and combine the new data frame we processed here 
    
train = pd.concat(trains)
test = pd.concat(tests)


In [33]:
test = test.resample('D').mean()
train = train.resample('D').mean()

In [30]:
train.shape

(1121, 1)

In [29]:
test.shape

(281, 1)

### Individual Stations not Concatenated, callable from a list
This will allow us to predict on individual sites (stations) if time allows, or after this project, we would like to investigate each site further. 

In [19]:
# From the DATA folder, we wil run through all of the individual csv files. 

path = r'DATA/' 
allFiles = glob.glob(path + '/*.csv')

# Prep the test and train data frames.

test = pd.DataFrame()
train = pd.DataFrame()
all_trains = []
all_tests = []

for file_ in allFiles:
    
    # Read in and set data index. 
    df = pd.read_csv(file_,index_col = None,header = 0)
    df['Date'] = pd.to_datetime(df[['year','month','day','hour']])
    df = df.set_index('Date')
    #Limits to end 2016 
    df = df.loc[:'2016-12-31 23:00:00']
    
    # Drop unwanted columns, in univariate we should only have the date and the target. 
    df = df.drop(columns=['year','month','day','hour','No', 
                         'PM10', 'SO2', 'NO2', 'CO', 'O3', 
                         'TEMP', 'PRES', 'DEWP', 'RAIN','wd', 'WSPM'], axis = 1)
    
    
    
    
    # Train test split to prevent data leakage
    train = df[:int(df.shape[0]*0.8)]
    test = df[int(df.shape[0]*0.8):]
     
    
    # Interpolate the train data and add to the rest of the train list 
    
    train = train.interpolate(method = 'time')
    all_trains.append(train)
    
    # Interpolate the test data and add to the rest of the test list
    
    test = test.interpolate(method = 'time')
    test['PM2.5'] = test['PM2.5'].fillna(method = 'bfill')
    all_tests.append(test)

In [44]:
Gucheng = all_tests[0]
Huairou = all_tests[1]
Tiantan = all_tests[2]
Changping = all_tests[3]
Guanyuan = all_tests[4]
Nongzhanguan = all_tests[5]
Wanliu = all_tests[6]
Dongsi = all_tests[7]
Wanshouxigong = all_tests[8]
Aotizhongxin = all_tests[9]
Dingling = all_tests[10]
Shunyi = all_tests[11]

trainGucheng = all_trains[0]
trainHuairou = all_trains[1]
trainTiantan = all_trains[2]
trainChangping = all_trains[3]
trainGuanyuan = all_trains[4]
trainNongzhanguan = all_trains[5]
trainWanliu = all_trains[6]
trainDongsi = all_trains[7]
trainWanshouxigong = all_trains[8]
trainAotizhongxin = all_trains[9]
trainDingling = all_trains[10]
trainShunyi = all_trains[11]

### Pickle the Time DF and the Train and Test DFs, then proceed to the EDA Notebook
The Time DF will be used for visualizations.
The Train Test Split will be used in Predictions as well as for comparative visuals later on. 
The sites will be used at a later date. 

In [34]:
test.to_pickle('PKL/test.pkl')
train.to_pickle('PKL/train.pkl')
time.to_pickle('PKL/time.pkl')

In [ ]:
# Station Sites 

Aotizhongxin.to_pickle('PKL/Aotizhongxin.pkl')
Changping.to_pickle('PKL/Changping.pkl')
Dingling.to_pickle('PKL/Dingling.pkl')
Dongsi.to_pickle('PKL/Dongsi.pkl')
Guanyuan.to_pickle('PKL/Guanyuan.pkl')
Gucheng.to_pickle('PKL/Gucheng.pkl')
Huairou.to_pickle('PKL/Huairou.pkl')
Nongzhanguan.to_pickle('PKL/Nongzhanguan.pkl')
Shunyi.to_pickle('PKL/Shunyi.pkl')
Tiantan.to_pickle('PKL/Tiantan.pkl')
Wanliu.to_pickle('PKL/Wanliu.pkl')
Wanshouxigong.to_pickle('PKL/Wanshouxigong.pkl')

trainGucheng.to_pickle('PKL/trainGucheng.pkl')
trainHuairou.to_pickle('PKL/trainHuairou.pkl')
trainTiantan.to_pickle('PKL/trainTiantan.pkl')
trainChangping.to_pickle('PKL/trainChangping.pkl')
trainGuanyuan.to_pickle('PKL/trainGuanyuan.pkl')
trainNongzhanguan.to_pickle('PKL/trainNongzhanguan.pkl')
trainWanliu.to_pickle('PKL/trainWanliu.pkl')
trainDongsi.to_pickle('PKL/trainDongsi.pkl')
trainWanshouxigong.to_pickle('PKL/trainWanshouxigong.pkl')
trainAotizhongxin.to_pickle('PKL/trainAotizhongxin.pkl')
trainDingling.to_pickle('PKL/trainDingling.pkl')
trainShunyi.to_pickle('PKL/trainShunyi.pkl')